In [2]:
# Import necessary modules from Flask
from flask import Flask, request
import os

# Import required components from LlamaIndex
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    PromptTemplate,
)
from llama_index.readers.file import CSVReader # For reading CSV files
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import (
    SentenceSplitter,
    SemanticSplitterNodeParser, # For semantic splitting of nodes
)
from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core import (
    load_index_from_storage,
    StorageContext,
)
from llama_index.core.evaluation import DatasetGenerator, RelevancyEvaluator
from dotenv import load_dotenv


load_dotenv(dotenv_path="./variable.env")

# Retrieve the OpenAI API key and paths from environment variables
api_key = os.getenv("OPENAI_API_KEY")
# Initialize OpenAI language model
llm = OpenAI(model="gpt-3.5-turbo")



# Define the prompt template for query answering
text_qa_template_str = (
    "Context information is"
    " below.\n---------------------\n{context_str}\n---------------------\nUsing"
    "You are an assistant for question-answering based on a certain context.\n"
    "- Use the following context to answer the question.\n"
    "- Don't tell any information that is not mentionned in the context.\n"
    "- Don't mention the use of the context in your answers.\n"
    "- Never refer to the context in your answer.\n"
    "- Use Three sentences maximum and keep the answer concise.\n"
    "- Use the context provided as a basis for a complete answer.\n" 
    "- Always respond in the same language as the question.\n "
    "- If someone asks for a location, always include the Google Maps link to the location in the answer.\n"
    "- If it is not possible to provide an answer based on the context, don't invent an answer and just say in the language of the question: 'Sorry, I couldn't answer your question based on my knowledge. Can you please rephrase it or ask another question?'"
    "- Do not hallucinate or invent information not present in the context.\n"
    "- If the context contains links to other websites, please include the links as well.\n"   
    "- Using the context , answer"
    " the question: {query_str}\n"
    "- Do not mention or refer to the context, 'context information', 'provided information', or any similar terms in your answers.\n"
    "- Always give a detailed answer that encompasses all the information.\n"
    "- If it is not possible to provide an answer based on the context, don't invent an answer and just say in the language of the question: 'Sorry, I couldn't answer your question based on my knowledge. Can you please rephrase it or ask another question?'\n"
)
text_qa_template = PromptTemplate(text_qa_template_str)



        
storage_context = StorageContext.from_defaults(persist_dir=f"C:\\Users\\Lenovo\\Desktop\\Code for production\\Stockage\\8c9796ee-28f4-4994-bc2c-ead53c32ae89")
index = load_index_from_storage(storage_context=storage_context)


    # Create retriever for the vector index

retriever = VectorIndexRetriever(
        index=index,
        similarity_top_k=5,
        llm=llm
    )


    # configure response synthesizer
response_synthesizer = get_response_synthesizer(text_qa_template=text_qa_template)

   
        # Create query engine with index and template
query_engine = RetrieverQueryEngine(
      retriever=retriever,
      response_synthesizer=response_synthesizer,
    )

In [3]:
response = query_engine.query("Where is the hotel located?")
print(response)

The hotel is located on Mohamed 6 avenue in the Hivernage area. For the exact location on Google Maps, you can visit: https://maps.app.goo.gl/CKjSrpgXRTK5Ynkc8. It is situated in a luxury quarter, just a 10-minute drive from the city center.
